<a href="https://colab.research.google.com/github/danielmlow/tutorials/blob/main/text/deidentify_pii.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Tutorials: https://microsoft.github.io/presidio/tutorial/

You might want to download other languages


In [ ]:
!pip install presidio-analyzer
!pip install presidio-anonymizer
!python -m spacy download en_core_web_lg # you can download other languages

# Text in txt files


In [ ]:

from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from tqdm import tqdm



anonymized_text_all = []

path = './data/transcripts/'

output_dir = './data/deidentified_transcripts/'

transcript_paths = []

for root, dirs, files in os.walk(path):
    # Skip hidden directories
    dirs[:] = [d for d in dirs if not d.startswith('.')]

    for file in files:
        if file.endswith('_transcript'):
            full_path = os.path.join(root, file)
            # Convert to forward slashes for consistency
            full_path = full_path.replace(os.sep, '/')
            transcript_paths.append(full_path)


# loop through txt files:

for path in transcript_paths:
  with open(path + path, 'r') as f:
    text = f.read()

    # Set up the engine, loads the NLP module (spaCy model by default)
    # and other PII recognizers
    analyzer = AnalyzerEngine()

    # Call analyzer to get results
    results = analyzer.analyze(text=text,
                            entities=["PERSON", "PHONE_NUMBER","NUMBER"], #You could add LOCATION
                            language='en')
    # print(results)

    # Analyzer results are passed to the AnonymizerEngine for anonymization

    anonymizer = AnonymizerEngine()

    anonymized_text = anonymizer.anonymize(text=text,analyzer_results=results)



    # Save deidentified txt in same folder

    # Option A
    # paht = path.replace('.txt', '')
    # with open(path + '_deidentified.txt', 'w') as f:

    # Option B:
    filename = path.replace('.txt', '').split('/')[-1]

    with open(output_dir + filename + '_deidentified.txt', 'w') as f:
        f.write(anonymized_text.text)





# If you have a DF

In [ ]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from tqdm import tqdm



anonymized_text_all = []


for text in tqdm(df['Transcript'].tolist()):


    # Set up the engine, loads the NLP module (spaCy model by default)
    # and other PII recognizers
    analyzer = AnalyzerEngine()

    # Call analyzer to get results
    results = analyzer.analyze(text=text,
                            entities=["PERSON","NAME", "PHONE_NUMBER","NUMBER"], #You could add LOCATION
                            language='en')
    # print(results)

    # Analyzer results are passed to the AnonymizerEngine for anonymization

    anonymizer = AnonymizerEngine()

    anonymized_text = anonymizer.anonymize(text=text,analyzer_results=results)
    anonymized_text_all.append(text)

df['Transcript_deidentified'] = anonymized_text_all
